<a href="https://colab.research.google.com/github/JGrant3101/ME4MachineLearning/blob/main/Coursework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Will start by importing all the libraries which might need to be used.

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn import datasets
from sklearn import model_selection
from sklearn import naive_bayes
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import KFold
import pandas
import plotly.graph_objects as go
import sys
import tensorflow
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import *

Firstly going to build the neural network for dataset1

In [10]:
# Start by importing the dataset
dataset1 = pandas.read_csv('/content/dataset1.csv')

# Write each column of the file to an array to make it easier to work with
ArmLength1 = np.array(dataset1['Arm length (m)'][:])
BallWeight1 = np.array(dataset1['Ball weight (kg)'][:])
BallRadius1 = np.array(dataset1['Ball radius (mm)'][:])
AirTemp1 = np.array(dataset1['Air temperature (deg C)'][:])
SpringConst1 = np.array(dataset1['Spring constant (N per m)'][:])
DeviceWeight1 = np.array(dataset1['Device weight (kg)'][:])
TargetHit1 = np.array(dataset1['Target hit'][:])
# Converting our currently 1D y values to 2D to match our model
TargetHit1_binary = to_categorical(TargetHit1)

# Going to scale each all of the data so want to find the mean and standard deviation
ArmLength1Mean = np.mean(ArmLength1)
ArmLength1std = np.std(ArmLength1)
BallWeight1Mean = np.mean(BallWeight1)
BallWeight1std = np.std(BallWeight1)
BallRadius1Mean = np.mean(BallRadius1)
BallRadius1std = np.std(BallRadius1)
AirTemp1Mean = np.mean(AirTemp1)
AirTemp1std = np.std(AirTemp1)
SpringConst1Mean = np.mean(SpringConst1)
SpringConst1std = np.std(SpringConst1)
DeviceWeight1Mean = np.mean(DeviceWeight1)
DeviceWeight1std = np.std(DeviceWeight1)

# Finally creating the scaled arrays using these mean and std values
ArmLength1Scaled = (ArmLength1 - ArmLength1Mean)/ArmLength1std
BallWeight1Scaled = (BallWeight1 - BallWeight1Mean)/BallWeight1std
BallRadius1Scaled = (BallRadius1 - BallRadius1Mean)/BallRadius1std
AirTemp1Scaled = (AirTemp1 - AirTemp1Mean)/AirTemp1std
SpringConst1Scaled = (SpringConst1 - SpringConst1Mean)/SpringConst1std
DeviceWeight1Scaled = (DeviceWeight1 - DeviceWeight1Mean)/DeviceWeight1std

# Finally forming these into one array to be parsed into the fit function
Inputs1 = np.zeros((2000, 6))
Inputs1[:,0] = ArmLength1Scaled
Inputs1[:,1] = BallWeight1Scaled
Inputs1[:,2] = BallRadius1Scaled
Inputs1[:,3] = AirTemp1Scaled
Inputs1[:,4] = SpringConst1Scaled
Inputs1[:,5] = DeviceWeight1Scaled

Above we have imported and processed our input data to be used to train the network

Now will do the actual construction of the network

In [11]:
# Initialising our neural network
model1 = Sequential()

# Adding layers, these layers are two layers of 4 nodes using the ReLU function and 1 with 2 nodes using the softmax function
model1.add(Dense(units = 10, activation = 'relu', input_dim = 6))
model1.add(Dense(units = 12, activation = 'relu'))
model1.add(Dense(units = 6, activation = 'relu'))
model1.add(Dense(units = 2, activation = 'softmax'))

# Compiling the model
model1.compile(loss = 'categorical_crossentropy', optimizer = 'sgd')

Above is the construction of the neural network, as we have 2000 points I think we can do a 5 split K fold cross validation and get good results so going to run that below in order to see what results the model gives and be able to work on improving it. This method is important to ensure that minimal overfitting is ocurring.

In [12]:
# Setting up a k fold cross validation
kf1 = KFold(n_splits = 5, shuffle = True)

# Defining empty arrays to add our fraction values to so an average can be calculated at the end
testfractions1 = 0
trainfractions1 = 0

for train_index, test_index in kf1.split(Inputs1):
  TrainingInputs = Inputs1[train_index]
  TrainingTargetHit = TargetHit1_binary[train_index]
  TestInputs = Inputs1[test_index]
  TestTargetHit = TargetHit1_binary[test_index]

  # Use X_train, y_train to train the SVM
  model1.fit(TrainingInputs, TrainingTargetHit, epochs = 10, batch_size = 40)
  # Use svm.predict() to predict the output for the test data set
  testresults = model1.predict(TestInputs)
  # loop through to compare the test data output to what it should be and obtain the fraction of correct classifications
  count = 0
  for i in range(len(TestTargetHit[:,0])):
    if (testresults[i, 0] > testresults[i, 1] and TestTargetHit[i, 0] > TestTargetHit[i, 1]) or (testresults[i, 0] < testresults[i, 1] and TestTargetHit[i, 0] < TestTargetHit[i, 1]):
      count = count+1
  testfractions1 = testfractions1 + (count / len(TestTargetHit[:,0]))
  # Do the same prediction and assessment performance with the training data
  count = 0
  trainresults = model1.predict(TrainingInputs)
  for i in range(len(TrainingTargetHit[:,0])):
    if (trainresults[i, 0] > trainresults[i, 1] and TrainingTargetHit[i, 0] > TrainingTargetHit[i, 1]) or (trainresults[i, 0] < trainresults[i, 1] and TrainingTargetHit[i, 0] < TrainingTargetHit[i, 1]):
      count = count+1
  trainfractions1 = trainfractions1 + (count / len(TrainingTargetHit[:,0]))

# Dividing the sum of 5 fraction values to get the average
testfraction1 = testfractions1/5
trainfraction1 = trainfractions1/5

# Printing the final accuracy values
print(testfraction1)
print(trainfraction1)

Epoch 1/10
40/40 [==============================] - 1s 2ms/step - loss: 0.7342
Epoch 2/10
40/40 [==============================] - 0s 2ms/step - loss: 0.6592
Epoch 3/10
40/40 [==============================] - 0s 2ms/step - loss: 0.6328
Epoch 4/10
40/40 [==============================] - 0s 2ms/step - loss: 0.6116
Epoch 5/10
40/40 [==============================] - 0s 2ms/step - loss: 0.5905
Epoch 6/10
40/40 [==============================] - 0s 2ms/step - loss: 0.5689
Epoch 7/10
40/40 [==============================] - 0s 2ms/step - loss: 0.5476
Epoch 8/10
40/40 [==============================] - 0s 2ms/step - loss: 0.5267
Epoch 9/10
40/40 [==============================] - 0s 2ms/step - loss: 0.5061
Epoch 10/10
50/50 [==============================] - 0s 1ms/step
Epoch 1/10
40/40 [==============================] - 0s 2ms/step - loss: 0.4564
Epoch 2/10
40/40 [==============================] - 0s 2ms/step - loss: 0.4374
Epoch 3/10
40/40 [==============================] - 0s 2ms/step - 

Now model has been tested and proven out can export and run through the check script provided to check everything works and also get an idea of how accurate it is.

In [13]:
# Exporting the model
model1.save('jg619-1.h5')

# Saving the scaling parameters for temperature and strength
scArray = np.array([[ArmLength1Mean, BallWeight1Mean, BallRadius1Mean, AirTemp1Mean, SpringConst1Mean, DeviceWeight1Mean], [ArmLength1std, BallWeight1std, BallRadius1std, AirTemp1std, SpringConst1std, DeviceWeight1std]])
np.savetxt('jg619-1.txt', scArray)

# Downloading the files
from google.colab import files
files.download('jg619-1.h5')
files.download('jg619-1.txt')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Above is all the code required for dataset 1, will copy the code blocks and replicate them below for dataset 2

In [14]:
# Start by importing the dataset
dataset2 = pandas.read_csv('/content/dataset2.csv')

# Write each column of the file to an array to make it easier to work with
ArmLength2 = np.array(dataset2['Arm length (m)'][:])
BallWeight2 = np.array(dataset2['Ball weight (kg)'][:])
BallRadius2 = np.array(dataset2['Ball radius (mm)'][:])
AirTemp2 = np.array(dataset2['Air temperature (deg C)'][:])
SpringConst2 = np.array(dataset2['Spring constant (N per m)'][:])
DeviceWeight2 = np.array(dataset2['Device weight (kg)'][:])
TargetHit2 = np.array(dataset2['Target hit'][:])
# Converting our currently 1D y values to 2D to match our model
TargetHit2_binary = to_categorical(TargetHit2)

# Going to scale each all of the data so want to find the mean and standard deviation
ArmLength2Mean = np.mean(ArmLength2)
ArmLength2std = np.std(ArmLength2)
BallWeight2Mean = np.mean(BallWeight2)
BallWeight2std = np.std(BallWeight2)
BallRadius2Mean = np.mean(BallRadius2)
BallRadius2std = np.std(BallRadius2)
AirTemp2Mean = np.mean(AirTemp2)
AirTemp2std = np.std(AirTemp2)
SpringConst2Mean = np.mean(SpringConst2)
SpringConst2std = np.std(SpringConst2)
DeviceWeight2Mean = np.mean(DeviceWeight2)
DeviceWeight2std = np.std(DeviceWeight2)

# Finally creating the scaled arrays using these mean and std values
ArmLength2Scaled = (ArmLength2 - ArmLength2Mean)/ArmLength2std
BallWeight2Scaled = (BallWeight2 - BallWeight2Mean)/BallWeight2std
BallRadius2Scaled = (BallRadius2 - BallRadius2Mean)/BallRadius2std
AirTemp2Scaled = (AirTemp2 - AirTemp2Mean)/AirTemp2std
SpringConst2Scaled = (SpringConst2 - SpringConst2Mean)/SpringConst2std
DeviceWeight2Scaled = (DeviceWeight2 - DeviceWeight2Mean)/DeviceWeight2std

# Finally forming these into one array to be parsed into the fit function
Inputs2 = np.zeros((4000, 6))
Inputs2[:,0] = ArmLength2Scaled
Inputs2[:,1] = BallWeight2Scaled
Inputs2[:,2] = BallRadius2Scaled
Inputs2[:,3] = AirTemp2Scaled
Inputs2[:,4] = SpringConst2Scaled
Inputs2[:,5] = DeviceWeight2Scaled

In [15]:
# Initialising our neural network
model2 = Sequential()

# Adding layers, these layers are two layers of 4 nodes using the ReLU function and 1 with 2 nodes using the softmax function
model2.add(Dense(units = 10, activation = 'relu', input_dim = 6))
model2.add(Dense(units = 12, activation = 'relu'))
model2.add(Dense(units = 6, activation = 'relu'))
model2.add(Dense(units = 2, activation = 'softmax'))

# Compiling the model
model2.compile(loss = 'categorical_crossentropy', optimizer = 'sgd')

In [16]:
# Setting up a k fold cross validation
kf2 = KFold(n_splits = 5, shuffle = True)

# Defining empty arrays to add our fraction values to so an average can be calculated at the end
testfractions2 = 0
trainfractions2 = 0

for train_index, test_index in kf2.split(Inputs2):
  TrainingInputs = Inputs2[train_index]
  TrainingTargetHit = TargetHit2_binary[train_index]
  TestInputs = Inputs2[test_index]
  TestTargetHit = TargetHit2_binary[test_index]

  # Use X_train, y_train to train the SVM
  model2.fit(TrainingInputs, TrainingTargetHit, epochs = 10, batch_size = 40)
  # Use svm.predict() to predict the output for the test data set
  testresults = model2.predict(TestInputs)
  # loop through to compare the test data output to what it should be and obtain the fraction of correct classifications
  count = 0
  for i in range(len(TestTargetHit[:,0])):
    if (testresults[i, 0] > testresults[i, 1] and TestTargetHit[i, 0] > TestTargetHit[i, 1]) or (testresults[i, 0] < testresults[i, 1] and TestTargetHit[i, 0] < TestTargetHit[i, 1]):
      count = count+1
  testfractions2 = testfractions2 + (count / len(TestTargetHit[:,0]))
  # Do the same prediction and assessment performance with the training data
  count = 0
  trainresults = model1.predict(TrainingInputs)
  for i in range(len(TrainingTargetHit[:,0])):
    if (trainresults[i, 0] > trainresults[i, 1] and TrainingTargetHit[i, 0] > TrainingTargetHit[i, 1]) or (trainresults[i, 0] < trainresults[i, 1] and TrainingTargetHit[i, 0] < TrainingTargetHit[i, 1]):
      count = count+1
  trainfractions2 = trainfractions2 + (count / len(TrainingTargetHit[:,0]))

# Dividing the sum of 5 fraction values to get the average
testfraction2 = testfractions2/5
trainfraction2 = trainfractions2/5

# Printing the final accuracy values
print(testfraction2)
print(trainfraction2)

Epoch 1/10
80/80 [==============================] - 1s 2ms/step - loss: 0.6946
Epoch 2/10
80/80 [==============================] - 0s 2ms/step - loss: 0.6915
Epoch 3/10
80/80 [==============================] - 0s 2ms/step - loss: 0.6904
Epoch 4/10
80/80 [==============================] - 0s 2ms/step - loss: 0.6895
Epoch 5/10
80/80 [==============================] - 0s 2ms/step - loss: 0.6889
Epoch 6/10
80/80 [==============================] - 0s 2ms/step - loss: 0.6884
Epoch 7/10
80/80 [==============================] - 0s 2ms/step - loss: 0.6879
Epoch 8/10
80/80 [==============================] - 0s 2ms/step - loss: 0.6874
Epoch 9/10
80/80 [==============================] - 0s 2ms/step - loss: 0.6870
Epoch 10/10
100/100 [==============================] - 0s 2ms/step
Epoch 1/10
80/80 [==============================] - 0s 3ms/step - loss: 0.6864
Epoch 2/10
80/80 [==============================] - 0s 2ms/step - loss: 0.6857
Epoch 3/10
80/80 [==============================] - 0s 2ms/step 

In [17]:
# Exporting the model
model2.save('jg619-2.h5')

# Saving the scaling parameters for temperature and strength
scArray = np.array([[ArmLength2Mean, BallWeight2Mean, BallRadius2Mean, AirTemp2Mean, SpringConst2Mean, DeviceWeight2Mean], [ArmLength2std, BallWeight2std, BallRadius2std, AirTemp2std, SpringConst2std, DeviceWeight2std]])
np.savetxt('jg619-2.txt', scArray)

# Downloading the files
from google.colab import files
files.download('jg619-2.h5')
files.download('jg619-2.txt')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>